<a href="https://colab.research.google.com/github/robertoarturomc/ProgramacionConcurrente/blob/main/16_Multiprocessing_en_Python_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Programación Concurrente
## 16. Multiprocessing en Python II

¿Qué sucede si necesitamos crear varios procesos para manejar tareas que consumen más CPU? ¿Siempre debemos iniciar y esperar explícitamente a que finalicen? La solución aquí es usar la clase `Pool`.





In [1]:
import multiprocessing
import time
import math

In [2]:
# ¿Qué pasa si queremos calcular los cubos de todos los números, del 1 al 5,000,000?

N = 5000000

def cube(x):
    return math.sqrt(x)

with multiprocessing.Pool() as pool:
  result = pool.map(cube, range(10,N))
  print("¡Programa Terminado!")

¡Programa Terminado!


Repitámoslo de nuevo; primero de manera Secuencial y de manera Concurrente con Multiprocesos (usando Pool).

In [3]:
start = time.time()

for i in range(10, N):
  cube(i)

print("Programa Secuencial Terminado")

end = time.time()

print("Se tardó ", end-start, "segundos")

Programa Secuencial Terminado
Se tardó  1.3141217231750488 segundos


In [10]:
start = time.time()

with multiprocessing.Pool() as pool:
  result = pool.map(cube, range(10,N))
  print("¡Programa Terminado!")

print("Programa Concurrente con Multi-procesos Terminado")

end = time.time()

print("Se tardó ", end-start, "segundos")

¡Programa Terminado!
Programa Concurrente con Multi-procesos Terminado
Se tardó  1.2114038467407227 segundos


El número de procesos que tu computadora es capaz de soportar depende del número de Núcelos que tu Procesador (CPU) tiene.

Por default,  `Pool` crea ese mayor número posible de procesos, dependiendo de mis núcleos. Pero, puedo darle un valor con el parámetro *processes*.


In [7]:
def fib(n):
    return n if n < 2 else fib(n - 2) + fib(n - 1)

with multiprocessing.Pool(processes=4) as pool:
  results = pool.map(fib, range(40))
  for i, result in enumerate(results):
      print(f"fib({i}) = {result}")

fib(0) = 0
fib(1) = 1
fib(2) = 1
fib(3) = 2
fib(4) = 3
fib(5) = 5
fib(6) = 8
fib(7) = 13
fib(8) = 21
fib(9) = 34
fib(10) = 55
fib(11) = 89
fib(12) = 144
fib(13) = 233
fib(14) = 377
fib(15) = 610
fib(16) = 987
fib(17) = 1597
fib(18) = 2584
fib(19) = 4181
fib(20) = 6765
fib(21) = 10946
fib(22) = 17711
fib(23) = 28657
fib(24) = 46368
fib(25) = 75025
fib(26) = 121393
fib(27) = 196418
fib(28) = 317811
fib(29) = 514229
fib(30) = 832040
fib(31) = 1346269
fib(32) = 2178309
fib(33) = 3524578
fib(34) = 5702887
fib(35) = 9227465
fib(36) = 14930352
fib(37) = 24157817
fib(38) = 39088169
fib(39) = 63245986


Recuerda que arrancar un Proceso toma tiempo. Al crear un proceso, el sistema operativo debe copiar memoria, cargar recursos y crear un nuevo entorno de ejecución (con su propio espacio de memoria, variables, etc.).
¡Arrancar un proceso es más pesado que crear un hilo!

In [19]:
# executor_map.py
from concurrent.futures import ProcessPoolExecutor, as_completed
from math import factorial
from time import perf_counter

def tarea(n: int) -> int:
    # Algo CPU-bound (factorial grande)
    return sum(int(d) for d in str(factorial(n)))

if __name__ == "__main__":
    nums = list(range(30_000, 30_100))  # 100 tareas
    t0 = perf_counter()
    results = {}
    with ProcessPoolExecutor() as ex:
        futs = {ex.submit(tarea, n): n for n in nums}
        for fut in as_completed(futs):
            n = futs[fut]
            results[n] = fut.result()
            # aquí podrías reportar progreso

    print(f"Ejemplos: {list(results.items())[:3]}")
    print(f"Total tareas: {len(results)}  |  Tiempo: {perf_counter()-t0:.2f}s")

ValueError: Exceeds the limit (2000 digits) for integer string conversion; use sys.set_int_max_str_digits() to increase the limit

In [18]:
import sys

sys.set_int_max_str_digits(2000)

Por cierto, en Python es buena idea usar la condición:

```
if __name__ == "__main__":
```

Esto porque evita que el código se ejecute varias veces. Sin ese bloque, el código que crea los procesos se ejecutaría otra vez dentro de cada proceso hijo.



Por cierto, para ver cuántos núcleos tiene tu Procesador, presiona Ctrl + Alt + Del (Supr) para abrir el Administrador de Tareas en WIndows.


### Tarea

Leer este artículo:

https://tonybaloney.github.io/posts/why-isnt-python-async-more-popular.html